In [1]:
!pip install --upgrade botocore==1.23.26
!pip install --upgrade urllib3==1.22.0
!pip install py7zr
!pip install s3fs
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartogether
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install geopandas
!pip install pygeos
!pip install geopy
!pip install pynsee[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.13
    Uninstalling botocore-1.29.13:
      Successfully uninstalled botocore-1.29.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.26.13 requires botocore<1.30.0,>=1.29.13, but you have botocore 1.23.26 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11
ERROR: pip's dependency resolver does not currently take into ac

In [1]:
import contextily as ctx
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

/opt/mamba/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [33]:
import pynsee
from pynsee.utils.init_conn import init_conn

N=1000

init_conn(insee_key="k32RC1ZJH8RV4Llh6kTRakU15tca", insee_secret="CTnVGRLchI7dbJOlrvYbrOyIfnMa")
data = pynsee.search_sirene(variable = ["activitePrincipaleEtablissement"],
                       pattern = "10.71C", kind = 'siret', number=N)

data = data.loc[data['dateFin'].isnull()]
df=data[["activitePrincipaleEtablissement","codePostalEtablissement","libelleCommuneEtablissement"]]

Token has been created
!!! Please subscribe to BDM API on api.insee.fr !!!
!!! Please subscribe to Metadata API on api.insee.fr !!!
!!! Please subscribe to Local Data API on api.insee.fr !!!
Locally saved data has been used
Set update=True to trigger an update


In [34]:
df=df.dropna()
liste_departements=["75","77","78","91","92","93","94","95"]
department=[]
for ville in df.codePostalEtablissement:
    department.append(ville[:2])
df['departement']=department
df=df[df.activitePrincipaleEtablissement=="10.71C"]
df=df[df.departement.isin(liste_departements)]
df

,activitePrincipaleEtablissement,codePostalEtablissement,libelleCommuneEtablissement,departement
5,10.71C,78600,MAISONS-LAFFITTE,78
12,10.71C,75004,PARIS 4,75
17,10.71C,75008,PARIS 8,75
18,10.71C,75005,PARIS 5,75
23,10.71C,92140,CLAMART,92
27,10.71C,92150,SURESNES,92
29,10.71C,75116,PARIS 16,75
37,10.71C,95100,ARGENTEUIL,95
38,10.71C,75011,PARIS 11,75
45,10.71C,93360,NEUILLY-PLAISANCE,93


In [35]:
def get_location(adress):
    geolocator = Nominatim(user_agent="tutorial")
    location = geolocator.geocode(adress)
    return((location.latitude, location.longitude))

def get_location_all(commerces):
    geom = []
    for k in range(len(commerces)):
        geom.append(Point(get_location(commerces.iloc[k,1]+" "+commerces.iloc[k,2])))
    commerces = commerces.set_geometry(geom)
    return(commerces)

In [36]:
df=get_location_all(df)
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
[<shapely.geometry.point.Point object at 0x7fdaf545d6c0>, <shapely.geometry.point.Point object at 0x7fdaeeee7d00>, <shapely.geometry.point.Point object at 0x7fdaf429c130>, <shapely.geometry.point.Point object at 0x7fdaf429c520>, <shapely.geometry.point.Point object at 0x7fdaf429f880>, <shapely.geometry.point.Point object at 0x7fdaf429fb80>, <shapely.geometry.point.Point object at 0x7fdaf429eaa0>, <shapely.geometry.point.Point object at 0x7fdaf429e0b0>, <shapely.geometry.point.Point object at 0x7fdaef9544c0>, <shapely.geometry.point.Point object at 0x7fdaef955480>, <shapely.geometry.point.Point object at 0x7fdaef955990>, <shapely.geometry.point.Point object at 0x7fdaef954a60>, <shapely.geometry.point.Point object at 0x7fdaef955510>, <shapely.geometry.point.Point object at 0x7fdaef9556f0>, <shapely.geometry.point.Point object at 0x7fdaef955000>, <shapely.geometry.point.Point object at 0x7fdaef954eb0>

,activitePrincipaleEtablissement,codePostalEtablissement,libelleCommuneEtablissement,departement,geometry
5,10.71C,78600,MAISONS-LAFFITTE,78,POINT (48.946 2.145)
12,10.71C,75004,PARIS 4,75,POINT (48.852 2.361)
17,10.71C,75008,PARIS 8,75,POINT (48.877 2.324)
18,10.71C,75005,PARIS 5,75,POINT (20.864 106.668)
23,10.71C,92140,CLAMART,92,POINT (48.800 2.263)
27,10.71C,92150,SURESNES,92,POINT (48.871 2.226)
29,10.71C,75116,PARIS 16,75,POINT (48.884 2.322)
37,10.71C,95100,ARGENTEUIL,95,POINT (48.948 2.248)
38,10.71C,75011,PARIS 11,75,POINT (48.817 2.357)
45,10.71C,93360,NEUILLY-PLAISANCE,93,POINT (48.861 2.506)
